### Import required libraries:


In [ ]:
import numpy as np
import pandas as pd
import talib
import MetaTrader5 as mt5

### Connect to the MetaTrader5 terminal:


In [ ]:
# Connect to the MetaTrader5 terminal
if not mt5.initialize():
    print("Failed to initialize MetaTrader5")
    mt5.shutdown()
    exit()

### Retrieve historical price data:

In [ ]:
# Specify the symbol and timeframe
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_H1  # 1-hour timeframe

# Retrieve historical price data
num_bars = 1000  # Number of bars to retrieve
rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(rates, columns=['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume'])
df['time'] = pd.to_datetime(df['time'], unit='s')

### Implement the Wavy Tunnel and other indicators:


In [ ]:
# Wavy Tunnel
df['wavy_h'] = talib.EMA(df['high'], timeperiod=34)
df['wavy_c'] = talib.EMA(df['close'], timeperiod=34)
df['wavy_l'] = talib.EMA(df['low'], timeperiod=34)
df['ema_12'] = talib.EMA(df['close'], timeperiod=12)
df['tunnel1'] = talib.EMA(df['close'], timeperiod=144)
df['tunnel2'] = talib.EMA(df['close'], timeperiod=169)

### Implement user inputs:


In [ ]:
# User inputs
apply_rsi_filter = True
apply_atr_filter = True
apply_threshold = True
threshold_values = {
    'USD': 2,
    'EUR': 2,
    'JPY': 300,
    'GBP': 6,
    'CHF': 2,
    'AUD': 2,
    'default': 100
}

# Liquidation level threshold
min_auto_tp_threshold_values = {
    'USD': 5,
    'EUR': 10,
    'JPY': 100,
    'GBP': 50,
    'CHF': 5,
    'AUD': 5,
    'default': 100
}

# Last take profit limit
last_tp_limit_values = {
    'USD': 15,
    'EUR': 10,
    'JPY': 800,
    'GBP': 60,
    'CHF': 15,
    'AUD': 15,
    'default': 250
}

peak_type = 21

enable_second_strategy = True
min_gap_second_values = {
    'USD': 15,
    'EUR': 15,
    'JPY': 650,
    'GBP': 50,
    'CHF': 15,
    'AUD': 15,
    'default': 650
}

max_allow_into_zone = 0.25

is_range = False
from_date = '2024-02-01'
to_date = '2024-03-16'

is_proximity = True
support_resistance_proximity = 0.01

rsi_period = 14
rsi_upper = 70
rsi_lower = 30

atr_period = 14
atr_multiplier = 2

tp_lot_percent = [60, 15, 10, 10]
tp_weights = [0.2, 0.4, 0.6, 0.8]

tp_lot_percent_second = [60, 15, 10, 10]
tp_weights_second = [0.2, 0.4, 0.6, 0.8]

max_allow_into_zone_second = 0.25

### Implement trend analysis:


In [ ]:
# Trend Analysis
df['long_term_ema'] = talib.EMA(df['close'], timeperiod=200)
df['rsi'] = talib.RSI(df['close'], timeperiod=14)
rsi_upper = 70
rsi_lower = 30

### Implement peak and dip detection:


In [ ]:
# Peak and Dip Detection
def detect_peaks_and_dips(df, peak_type):
    peaks = []
    dips = []
    for i in range(len(df)):
        if i < peak_type or i >= len(df) - peak_type:
            continue
        is_peak = True
        is_dip = True
        for j in range(peak_type):
            if df['high'][i] <= df['high'][i-j] or df['high'][i] <= df['high'][i+j]:
                is_peak = False
            if df['low'][i] >= df['low'][i-j] or df['low'][i] >= df['low'][i+j]:
                is_dip = False
        if is_peak:
            peaks.append(df['high'][i])
        if is_dip:
            dips.append(df['low'][i])
    return peaks, dips

peak_type = 21
peaks, dips = detect_peaks_and_dips(df, peak_type)

### Implement liquidation levels:


In [ ]:
# Liquidation Levels
def find_last_peak_dip(peaks, dips, current_price, min_threshold):
    last_peak = np.nan
    last_dip = np.nan
    for peak in reversed(peaks):
        if peak > current_price + min_threshold:
            last_peak = peak
            break
    for dip in reversed(dips):
        if dip < current_price - min_threshold:
            last_dip = dip
            break
    return last_peak, last_dip

min_threshold = 100  # Adjust this value based on the user input

### Implement entry conditions:

In [ ]:
def check_entry_conditions(row):
    buy_condition = (
        row['close'] > max(row['wavy_c'], row['wavy_h'], row['wavy_l']) and
        min(row['wavy_c'], row['wavy_h'], row['wavy_l']) > max(row['tunnel1'], row['tunnel2'])
    )
    sell_condition = (
        row['close'] < min(row['wavy_c'], row['wavy_h'], row['wavy_l']) and
        max(row['wavy_c'], row['wavy_h'], row['wavy_l']) < min(row['tunnel1'], row['tunnel2'])
    )

    if apply_rsi_filter:
        buy_condition &= row['rsi'] < rsi_upper
        sell_condition &= row['rsi'] > rsi_lower

    if apply_atr_filter:
        atr_value = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
        buy_condition &= row['close'] < row['long_term_ema'] - 2 * atr_value
        sell_condition &= row['close'] > row['long_term_ema'] + 2 * atr_value

    if apply_threshold:
        threshold = threshold_values.get(symbol[:3], threshold_values['default']) * mt5.symbol_info(symbol).trade_tick_size
        buy_condition &= row['close'] > max(row['wavy_c'], row['wavy_h'], row['wavy_l']) + threshold
        sell_condition &= row['close'] < min(row['wavy_c'], row['wavy_h'], row['wavy_l']) - threshold

    return buy_condition, sell_condition

df['buy_signal'], df['sell_signal'] = zip(*df.apply(check_entry_conditions, axis=1))

### Implement support and resistance proximity:


In [ ]:
# Support and Resistance Proximity
def check_support_resistance_proximity(row):
    support_level = talib.SMA(talib.MIN(df['low'], 18), 5)
    resistance_level = talib.SMA(talib.MAX(df['high'], 18), 5)
    proximity_threshold = 0.01  # Adjust this value based on user input

    near_support = row['close'] <= support_level * (1 + proximity_threshold)
    near_resistance = row['close'] >= resistance_level * (1 - proximity_threshold)

    return near_support, near_resistance

df['near_support'], df['near_resistance'] = zip(*df.apply(check_support_resistance_proximity, axis=1))



### Implement position management:


In [ ]:
# Position Management
def calculate_position_size(row):
    balance = mt5.account_info().balance
    risk_percent = 0.01  # Adjust this value based on user input
    position_size = (balance * risk_percent) / (row['close'] * mt5.symbol_info(symbol).trade_tick_size)
    return position_size

def open_position(row):
    if row['buy_signal'] and not row['near_resistance']:
        lot_size = calculate_position_size(row)
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_BUY,
            "price": row['close'],
            "magic": 12345,
            "comment": "Python Buy Order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        print(f"Buy Order Sent: {result}")

    elif row['sell_signal'] and not row['near_support']:
        lot_size = calculate_position_size(row)
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_SELL,
            "price": row['close'],
            "magic": 12345,
            "comment": "Python Sell Order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        print(f"Sell Order Sent: {result}")

def close_positions():
    positions = mt5.positions_get(symbol=symbol)
    for position in positions:
        if position.type == mt5.POSITION_TYPE_BUY:
            close_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": position.volume,
                "type": mt5.ORDER_TYPE_SELL,
                "position": position.ticket,
                "price": mt5.symbol_info_tick(symbol).bid,
                "magic": 12345,
                "comment": "Python Close Buy Order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_request)
            print(f"Closed Buy Position: {result}")

        elif position.type == mt5.POSITION_TYPE_SELL:
            close_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": position.volume,
                "type": mt5.ORDER_TYPE_BUY,
                "position": position.ticket,
                "price": mt5.symbol_info_tick(symbol).ask,
                "magic": 12345,
                "comment": "Python Close Sell Order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_request)
            print(f"Closed Sell Position: {result}")

### Implement the second strategy:


In [ ]:
# Second Strategy
def check_second_strategy_conditions(row):
    fully_crossed_above = (
        row['close'] > max(row['wavy_c'], row['wavy_h'], row['wavy_l']) and
        row['close'] < min(row['tunnel1'], row['tunnel2'])
    )
    fully_crossed_below = (
        row['close'] < min(row['wavy_c'], row['wavy_h'], row['wavy_l']) and
        row['close'] > max(row['tunnel1'], row['tunnel2'])
    )
    return fully_crossed_above, fully_crossed_below

df['second_buy_signal'], df['second_sell_signal'] = zip(*df.apply(check_second_strategy_conditions, axis=1))

### Implement backtesting and visualization:


In [ ]:
# Backtesting and Visualization
initial_balance = 10000
balance = initial_balance
position = None
trades = []

for _, row in df.iterrows():
    if position is None:
        if row['buy_signal']:
            position = 'long'
            entry_price = row['close']
            entry_time = row['time']
            lot_size = calculate_position_size(row)
            balance -= lot_size * entry_price
            trades.append(('buy', entry_time, entry_price, lot_size))
        elif row['sell_signal']:
            position = 'short'
            entry_price = row['close']
            entry_time = row['time']
            lot_size = calculate_position_size(row)
            balance += lot_size * entry_price
            trades.append(('sell', entry_time, entry_price, lot_size))
    else:
        if position == 'long' and (row['sell_signal'] or row['close'] < min(row['wavy_c'], row['wavy_h'], row['wavy_l'])):
            exit_price = row['close']
            exit_time = row['time']
            balance += lot_size * exit_price
            trades.append(('sell', exit_time, exit_price, lot_size))
            position = None
        elif position == 'short' and (row['buy_signal'] or row['close'] > max(row['wavy_c'], row['wavy_h'], row['wavy_l'])):
            exit_price = row['close']
            exit_time = row['time']
            balance -= lot_size * exit_price
            trades.append(('buy', exit_time, exit_price, lot_size))
            position = None

final_balance = balance
profit = final_balance - initial_balance
print(f"Initial Balance: {initial_balance}")
print(f"Final Balance: {final_balance}")
print(f"Profit: {profit}")

# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df['time'], df['close'], label='Price')
plt.plot(df['time'], df['wavy_h'], label='Wavy High')
plt.plot(df['time'], df['wavy_c'], label='Wavy Close')
plt.plot(df['time'], df['wavy_l'], label='Wavy Low')
plt.plot(df['time'], df['tunnel1'], label='Tunnel 1')
plt.plot(df['time'], df['tunnel2'], label='Tunnel 2')

buy_signals = df[df['buy_signal']]['close']
sell_signals = df[df['sell_signal']]['close']
plt.scatter(buy_signals.index, buy_signals, color='green', label='Buy Signal', marker='^')
plt.scatter(sell_signals.index, sell_signals, color='red', label='Sell Signal', marker='v')

plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Trading Signals')
plt.legend()
plt.grid(True)
plt.show()

### Disconnect from the MetaTrader5 terminal:


In [ ]:
# Disconnect from the MetaTrader5 terminal
mt5.shutdown()